In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:2000], X_train[2000:]
y_valid, y_train = y_train[:2000], y_train[2000:]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 10.0*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for i in [1, 2, 3]:
    for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20191110200258
0	Validation losses: 0.1582, 8.7629, 8.9211	Best Loss: 8.9211 (0)	Accuracy: 95.50%
1	Validation losses: 0.1215, 6.9447, 7.0662	Best Loss: 7.0662 (0)	Accuracy: 96.55%
2	Validation losses: 0.1017, 4.8316, 4.9333	Best Loss: 4.9333 (0)	Accuracy: 97.05%
3	Validation losses: 0.1215, 2.9567, 3.0782	Best Loss: 3.0782 (0)	Accuracy: 96.35%
4	Validation losses: 0.1074, 1.6165, 1.7239	Best Loss: 1.7239 (0)	Accuracy: 97.05%
5	Validation losses: 0.1232, 0.8252, 0.9484	Best Loss: 0.9484 (0)	Accuracy: 96.05%
6	Validation losses: 0.1088, 0.4323, 0.5410	Best Loss: 0.5410 (0)	Accuracy: 96.85%
7	Validation losses: 0.1264, 0.2680, 0.3944	Best Loss: 0.3944 (0)	Accuracy: 96.25%
8	Validation losses: 0.1215, 0.2127, 0.3342	Best Loss: 0.3342 (0)	Accuracy: 96.85%
9	Validation losses: 0.1345, 0.1983, 0.3328	Best Loss: 0.3328 (0)	Accuracy: 96.25%
10	Validation losses: 0.1329, 0.1936, 0.3265	Best Loss: 0.3265 (0)	Accuracy: 95.95%
11	Validation losses: 0.2590,

31	Validation losses: 0.1152, 0.1112, 0.2264	Best Loss: 0.2209 (1)	Accuracy: 96.75%
32	Validation losses: 0.1291, 0.1045, 0.2336	Best Loss: 0.2209 (2)	Accuracy: 96.40%
33	Validation losses: 0.0935, 0.1000, 0.1935	Best Loss: 0.1935 (0)	Accuracy: 97.20%
34	Validation losses: 0.0962, 0.0947, 0.1909	Best Loss: 0.1909 (0)	Accuracy: 97.40%
35	Validation losses: 0.0981, 0.0885, 0.1865	Best Loss: 0.1865 (0)	Accuracy: 97.45%
36	Validation losses: 0.0835, 0.0830, 0.1665	Best Loss: 0.1665 (0)	Accuracy: 97.80%
37	Validation losses: 0.0975, 0.0773, 0.1748	Best Loss: 0.1665 (1)	Accuracy: 97.60%
38	Validation losses: 0.0839, 0.0722, 0.1561	Best Loss: 0.1561 (0)	Accuracy: 97.90%
39	Validation losses: 0.0668, 0.0684, 0.1352	Best Loss: 0.1352 (0)	Accuracy: 98.15%
40	Validation losses: 0.0651, 0.0665, 0.1316	Best Loss: 0.1316 (0)	Accuracy: 98.35%
41	Validation losses: 0.0677, 0.0641, 0.1318	Best Loss: 0.1316 (1)	Accuracy: 98.40%
42	Validation losses: 0.0747, 0.0617, 0.1365	Best Loss: 0.1316 (2)	Accuracy:

0	Validation losses: 0.1561, 2.3838, 2.5399	Best Loss: 2.5399 (0)	Accuracy: 95.45%
1	Validation losses: 0.1210, 2.2550, 2.3759	Best Loss: 2.3759 (0)	Accuracy: 96.85%
2	Validation losses: 0.0948, 2.0682, 2.1630	Best Loss: 2.1630 (0)	Accuracy: 97.30%
3	Validation losses: 0.1112, 1.8384, 1.9497	Best Loss: 1.9497 (0)	Accuracy: 96.80%
4	Validation losses: 0.0917, 1.5851, 1.6768	Best Loss: 1.6768 (0)	Accuracy: 96.95%
5	Validation losses: 0.1294, 1.3264, 1.4558	Best Loss: 1.4558 (0)	Accuracy: 95.90%
6	Validation losses: 0.0906, 1.0786, 1.1692	Best Loss: 1.1692 (0)	Accuracy: 97.35%
7	Validation losses: 0.1090, 0.8542, 0.9632	Best Loss: 0.9632 (0)	Accuracy: 96.80%
8	Validation losses: 0.0925, 0.6626, 0.7551	Best Loss: 0.7551 (0)	Accuracy: 97.40%
9	Validation losses: 0.1165, 0.5067, 0.6232	Best Loss: 0.6232 (0)	Accuracy: 96.80%
10	Validation losses: 0.0906, 0.3866, 0.4772	Best Loss: 0.4772 (0)	Accuracy: 97.60%
11	Validation losses: 0.1087, 0.2993, 0.4080	Best Loss: 0.4080 (0)	Accuracy: 97.25%
12

29	Validation losses: 0.0777, 0.0823, 0.1600	Best Loss: 0.1600 (0)	Accuracy: 98.05%
30	Validation losses: 0.0867, 0.0776, 0.1643	Best Loss: 0.1600 (1)	Accuracy: 97.75%
31	Validation losses: 0.0891, 0.0724, 0.1615	Best Loss: 0.1600 (2)	Accuracy: 97.85%
32	Validation losses: 0.0673, 0.0664, 0.1337	Best Loss: 0.1337 (0)	Accuracy: 98.45%
33	Validation losses: 0.0640, 0.0603, 0.1243	Best Loss: 0.1243 (0)	Accuracy: 98.25%
34	Validation losses: 0.0615, 0.0553, 0.1168	Best Loss: 0.1168 (0)	Accuracy: 98.55%
35	Validation losses: 0.0611, 0.0513, 0.1125	Best Loss: 0.1125 (0)	Accuracy: 98.35%
36	Validation losses: 0.0612, 0.0484, 0.1095	Best Loss: 0.1095 (0)	Accuracy: 98.40%
37	Validation losses: 0.0626, 0.0463, 0.1089	Best Loss: 0.1089 (0)	Accuracy: 98.40%
38	Validation losses: 0.0607, 0.0449, 0.1056	Best Loss: 0.1056 (0)	Accuracy: 98.50%
39	Validation losses: 0.0609, 0.0440, 0.1049	Best Loss: 0.1049 (0)	Accuracy: 98.50%
40	Validation losses: 0.0613, 0.0436, 0.1048	Best Loss: 0.1048 (0)	Accuracy:

55	Validation losses: 0.0842, 0.0434, 0.1276	Best Loss: 0.0963 (5)	Accuracy: 97.25%
56	Validation losses: 0.0716, 0.0465, 0.1180	Best Loss: 0.0963 (6)	Accuracy: 97.95%
57	Validation losses: 0.1007, 0.0480, 0.1488	Best Loss: 0.0963 (7)	Accuracy: 97.10%
58	Validation losses: 0.0950, 0.0505, 0.1455	Best Loss: 0.0963 (8)	Accuracy: 97.65%
59	Validation losses: 0.1080, 0.0531, 0.1611	Best Loss: 0.0963 (9)	Accuracy: 97.40%
60	Validation losses: 0.0798, 0.0551, 0.1349	Best Loss: 0.0963 (10)	Accuracy: 97.90%
61	Validation losses: 0.0901, 0.0544, 0.1445	Best Loss: 0.0963 (11)	Accuracy: 97.95%
62	Validation losses: 0.0917, 0.0527, 0.1444	Best Loss: 0.0963 (12)	Accuracy: 98.00%
63	Validation losses: 0.0666, 0.0513, 0.1180	Best Loss: 0.0963 (13)	Accuracy: 98.40%
64	Validation losses: 0.0625, 0.0486, 0.1111	Best Loss: 0.0963 (14)	Accuracy: 98.50%
65	Validation losses: 0.0627, 0.0451, 0.1078	Best Loss: 0.0963 (15)	Accuracy: 98.40%
66	Validation losses: 0.0850, 0.0418, 0.1268	Best Loss: 0.0963 (16)	Ac

...

242	Validation losses: 0.0953, 0.0291, 0.1244	Best Loss: 0.1244 (0)	Accuracy: 98.25%
243	Validation losses: 0.0953, 0.0290, 0.1243	Best Loss: 0.1243 (0)	Accuracy: 98.25%
244	Validation losses: 0.0953, 0.0290, 0.1242	Best Loss: 0.1242 (0)	Accuracy: 98.25%
245	Validation losses: 0.0952, 0.0289, 0.1242	Best Loss: 0.1242 (0)	Accuracy: 98.25%
246	Validation losses: 0.0952, 0.0289, 0.1241	Best Loss: 0.1241 (0)	Accuracy: 98.25%
247	Validation losses: 0.0952, 0.0288, 0.1241	Best Loss: 0.1241 (0)	Accuracy: 98.25%
248	Validation losses: 0.0951, 0.0288, 0.1239	Best Loss: 0.1239 (0)	Accuracy: 98.25%
249	Validation losses: 0.0952, 0.0287, 0.1239	Best Loss: 0.1239 (0)	Accuracy: 98.25%
Total running time:	 19517
INFO:tensorflow:Restoring parameters from ./models/model_20191114094714.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    1.2199997902%
RANDOM SEED:   3 	LAMBDA_EXP:   -6.0
Now: 20191114151232


86	Validation losses: 0.0935, 0.0553, 0.1488	Best Loss: 0.1488 (0)	Accuracy: 98.15%
87	Validation losses: 0.0935, 0.0551, 0.1486	Best Loss: 0.1486 (0)	Accuracy: 98.15%
88	Validation losses: 0.0935, 0.0549, 0.1484	Best Loss: 0.1484 (0)	Accuracy: 98.15%
89	Validation losses: 0.0935, 0.0547, 0.1482	Best Loss: 0.1482 (0)	Accuracy: 98.15%
90	Validation losses: 0.0935, 0.0545, 0.1479	Best Loss: 0.1479 (0)	Accuracy: 98.15%
91	Validation losses: 0.0934, 0.0542, 0.1477	Best Loss: 0.1477 (0)	Accuracy: 98.15%
92	Validation losses: 0.0934, 0.0540, 0.1474	Best Loss: 0.1474 (0)	Accuracy: 98.15%
93	Validation losses: 0.0934, 0.0537, 0.1471	Best Loss: 0.1471 (0)	Accuracy: 98.15%
94	Validation losses: 0.0934, 0.0534, 0.1468	Best Loss: 0.1468 (0)	Accuracy: 98.15%
95	Validation losses: 0.0933, 0.0531, 0.1464	Best Loss: 0.1464 (0)	Accuracy: 98.15%
96	Validation losses: 0.0933, 0.0528, 0.1461	Best Loss: 0.1461 (0)	Accuracy: 98.15%
97	Validation losses: 0.0933, 0.0524, 0.1457	Best Loss: 0.1457 (0)	Accuracy:

183	Validation losses: 0.0926, 0.0406, 0.1332	Best Loss: 0.1332 (0)	Accuracy: 98.10%
184	Validation losses: 0.0926, 0.0405, 0.1330	Best Loss: 0.1330 (0)	Accuracy: 98.10%
185	Validation losses: 0.0926, 0.0404, 0.1329	Best Loss: 0.1329 (0)	Accuracy: 98.10%
186	Validation losses: 0.0926, 0.0403, 0.1329	Best Loss: 0.1329 (0)	Accuracy: 98.10%
187	Validation losses: 0.0926, 0.0402, 0.1328	Best Loss: 0.1328 (0)	Accuracy: 98.10%
188	Validation losses: 0.0925, 0.0401, 0.1327	Best Loss: 0.1327 (0)	Accuracy: 98.10%
189	Validation losses: 0.0925, 0.0401, 0.1326	Best Loss: 0.1326 (0)	Accuracy: 98.10%
190	Validation losses: 0.0925, 0.0400, 0.1325	Best Loss: 0.1325 (0)	Accuracy: 98.10%
191	Validation losses: 0.0925, 0.0399, 0.1324	Best Loss: 0.1324 (0)	Accuracy: 98.10%
192	Validation losses: 0.0925, 0.0398, 0.1324	Best Loss: 0.1324 (0)	Accuracy: 98.10%
193	Validation losses: 0.0925, 0.0398, 0.1323	Best Loss: 0.1323 (0)	Accuracy: 98.10%
194	Validation losses: 0.0925, 0.0397, 0.1322	Best Loss: 0.1322 (

Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    2.3899996281%
RANDOM SEED:   3 	LAMBDA_EXP:   -6.4
Now: 20191114212833
0	Validation losses: 0.1561, 0.0387, 0.1949	Best Loss: 0.1949 (0)	Accuracy: 95.55%
1	Validation losses: 0.1158, 0.0388, 0.1547	Best Loss: 0.1547 (0)	Accuracy: 96.85%
2	Validation losses: 0.1000, 0.0389, 0.1389	Best Loss: 0.1389 (0)	Accuracy: 97.20%
3	Validation losses: 0.1040, 0.0391, 0.1430	Best Loss: 0.1389 (1)	Accuracy: 96.95%
4	Validation losses: 0.1221, 0.0392, 0.1613	Best Loss: 0.1389 (2)	Accuracy: 96.60%
5	Validation losses: 0.1526, 0.0394, 0.1919	Best Loss: 0.1389 (3)	Accuracy: 96.15%
6	Validation losses: 0.0956, 0.0396, 0.1352	Best Loss: 0.1352 (0)	Accuracy: 97.65%
7	Validation losses: 0.1033, 0.0398, 0.1431	Best Loss: 0.1352 (1)	Accuracy: 97.45%
8	Validation losses: 0.1082, 0.0400, 0.1482	Best Loss: 0.1352 (2)	Accuracy: 97.20%
9	Validation losses: 0.1024, 0.0403, 0.1427	Best Loss: 0.1352 (3)	Accuracy: 97.55%
10	V

15	Validation losses: 0.1067, 0.0173, 0.1240	Best Loss: 0.1074 (11)	Accuracy: 97.20%
16	Validation losses: 0.1042, 0.0175, 0.1217	Best Loss: 0.1074 (12)	Accuracy: 97.85%
17	Validation losses: 0.0997, 0.0177, 0.1174	Best Loss: 0.1074 (13)	Accuracy: 97.65%
18	Validation losses: 0.1148, 0.0179, 0.1327	Best Loss: 0.1074 (14)	Accuracy: 97.70%
19	Validation losses: 0.1883, 0.0181, 0.2064	Best Loss: 0.1074 (15)	Accuracy: 96.00%
20	Validation losses: 0.0826, 0.0184, 0.1009	Best Loss: 0.1009 (0)	Accuracy: 97.80%
21	Validation losses: 0.1275, 0.0185, 0.1460	Best Loss: 0.1009 (1)	Accuracy: 97.30%
22	Validation losses: 0.1082, 0.0187, 0.1269	Best Loss: 0.1009 (2)	Accuracy: 97.65%
23	Validation losses: 0.1155, 0.0187, 0.1342	Best Loss: 0.1009 (3)	Accuracy: 97.75%
24	Validation losses: 0.1400, 0.0187, 0.1588	Best Loss: 0.1009 (4)	Accuracy: 98.00%
25	Validation losses: 0.1195, 0.0188, 0.1383	Best Loss: 0.1009 (5)	Accuracy: 97.95%
26	Validation losses: 0.1126, 0.0187, 0.1313	Best Loss: 0.1009 (6)	Accu